In [ ]:
import os
import sys
import re

import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import interpolate

sys.path.append(os.path.realpath('.'))
sys.path.append(os.path.join(os.path.expanduser('~'), 'bnelearn'))

from scripts.utils import *
from bnelearn.util.metrics import *

In [ ]:
COLORS = ("#0E6BA0", "#973D4C", "#30C0D2", "#0E6BA0", "#973D4C", "#30C0D2")
MARKERS = ("s", "D", "^")

In [ ]:
def split_mean_and_std_of_df_column(df, metric):
    df[[metric + ' mean', metric + ' std']] = df[metric].str.split(' ', 1, expand=True)
    df[metric + ' std'] = df[metric + ' std'].apply(lambda x: x[1:-1])
    df[metric + ' mean'] = df[metric + ' mean'].astype(float)
    df[metric + ' std'] = df[metric + ' std'].astype(float)
    return df[metric + ' mean'].to_numpy(), df[metric + ' std'].to_numpy()

---
## 1. Single item experiment

In [ ]:
metric = "eval_vs_bne/L_2"

paths = {
    "Smooth Market": "/home/kohring/bnelearn/experiments/smooth-new/single_item/first_price/uniform/symmetric/risk_neutral/2p/2022-07-29 Fri 12:10:34/full_results.csv",
    "NPGA": "/home/kohring/bnelearn/experiments/smooth-new/single_item/first_price/uniform/symmetric/risk_neutral/2p/2022-07-29 Fri 13:03:22/full_results.csv"
}

In [ ]:
fig, ax = plt.subplots(figsize=(4.5, 4))

for i, (key, path) in enumerate(paths.items()):
    df = pd.read_csv(path)
    df = df.loc[df['tag'] == metric]
    df = df.groupby(['epoch']).agg({'value': ['mean', 'std']})
    df.columns = ['means', 'stds']
    
    x = df.index.to_numpy()
    means, stds = df.means.to_numpy(), df.stds.to_numpy()

    plt.plot(x, means, "-" + MARKERS[i], label=key, color=COLORS[i], markevery=200)
    plt.fill_between(x, np.clip(means-stds, 0, 10), means+stds, color=COLORS[i], alpha=.1)

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.grid(which='major', axis='y', alpha=.5)

plt.xlabel('iteration'); plt.ylabel(ALIASES_LATEX[metric])
plt.xlim([-30, 2030])
plt.ylim([0.003, .31])
plt.legend(loc='upper right', ncol=2, framealpha=1, edgecolor="white")
plt.tight_layout()
plt.semilogy()
plt.savefig(path[:-40] + f"single_item_analysis.pdf")
plt.show()

In [ ]:
metrics = ["eval_vs_bne/L_2", 'meta/time_per_step']
paths = {
    "SM FPSB":        "/home/kohring/bnelearn/experiments/smooth-new/single_item/first_price/uniform/symmetric/risk_neutral/2p/2022-07-29 Fri 12:10:34",
    "NPGA FPSB":      "/home/kohring/bnelearn/experiments/smooth-new/single_item/first_price/uniform/symmetric/risk_neutral/2p/2022-07-29 Fri 13:03:22",
    "Reinforce FPSB": "/home/kohring/bnelearn/experiments/smooth-new/single_item/first_price/uniform/symmetric/risk_neutral/2p/2022-08-04 Thu 11:58:21",
    "SM SPSB":        "/home/kohring/bnelearn/experiments/smooth-new/single_item/second_price/uniform/symmetric/risk_neutral/2p/2022-07-29 Fri 12:29:05",
    "NPGA SPSB":      "/home/kohring/bnelearn/experiments/smooth-new/single_item/second_price/uniform/symmetric/risk_neutral/2p/2022-07-29 Fri 13:58:54",
    "Reinforce SPSB": "/home/kohring/bnelearn/experiments/smooth-new/single_item/second_price/uniform/symmetric/risk_neutral/2p/2022-08-04 Thu 12:24:00",
}

In [ ]:
aggregate_df = pd.DataFrame()
for i, (learner, path) in enumerate(paths.items()):
    df = multiple_exps_logs_to_df(
        path=path,
        metrics=metrics,
        with_setting_parameters=True
    )
    df = df[["Smoothing", "Pricing"] + [ALIASES_LATEX[m] for m in metrics]]
    df["Smoothing"] = df["Smoothing"].apply(lambda x: learner.split(" ")[0])
    df = df.rename(columns={"Smoothing": "Algo."})
    aggregate_df = pd.concat([aggregate_df, df])
aggregate_df.sort_values(["Pricing", "Algo."], inplace=True)
df_to_tex(aggregate_df, name='single_item_table.tex', label='tab:single_item', caption='Results of the single-item experiments.')
aggregate_df.head(10)

---
## 2. Simultaneous sale experiment

In [ ]:
# metrics = ["eval_vs_bne/L_2", 'meta/time_per_step']
metrics = ['eval/util_loss_ex_interim', "eval_vs_bne/L_2", 'meta/time_per_step']

paths = {
    "/home/kohring/bnelearn/experiments/smooth-2023-uneven-batch-sizes/single_item/first_price/uniform/symmetric/risk_neutral/2p/2022-12-27 Tue 12:22:03",
    "/home/kohring/bnelearn/experiments/smooth-2023-uneven-batch-sizes/single_item/first_price/uniform/symmetric/risk_neutral/2p/2022-12-27 Tue 12:22:53",
    "/home/kohring/bnelearn/experiments/smooth-2023-uneven-batch-sizes/single_item/first_price/uniform/symmetric/risk_neutral/2p/2022-12-27 Tue 12:24:10",
    "/home/kohring/bnelearn/experiments/smooth-2023-uneven-batch-sizes/single_item/first_price/uniform/symmetric/risk_neutral/2p/2022-12-27 Tue 14:18:46",
    "/home/kohring/bnelearn/experiments/smooth-2023-uneven-batch-sizes/single_item/first_price/uniform/symmetric/risk_neutral/2p/2022-12-27 Tue 14:26:50",
    "/home/kohring/bnelearn/experiments/smooth-2023-uneven-batch-sizes/single_item/first_price/uniform/symmetric/risk_neutral/2p/2022-12-27 Tue 14:34:00",
    "/home/kohring/bnelearn/experiments/smooth-2023-uneven-batch-sizes/single_item/first_price/uniform/symmetric/risk_neutral/2p/2022-12-27 Tue 19:14:04",
    "/home/kohring/bnelearn/experiments/smooth-2023-uneven-batch-sizes/single_item/first_price/uniform/symmetric/risk_neutral/2p/2022-12-27 Tue 19:42:28",
    "/home/kohring/bnelearn/experiments/smooth-2023-uneven-batch-sizes/single_item/first_price/uniform/symmetric/risk_neutral/2p/2022-12-27 Tue 19:51:46",
    "/home/kohring/bnelearn/experiments/smooth-large-scale/main/single_item/first_price/uniform/symmetric/risk_neutral/2p/2023-01-12 Thu 18:34:57",
    "/home/kohring/bnelearn/experiments/smooth-large-scale/main-ESPGLearner/single_item/first_price/uniform/symmetric/risk_neutral/2p/2023-01-13 Fri 11:06:04",
    "/home/kohring/bnelearn/experiments/smooth-large-scale/main-PGLearner/single_item/first_price/uniform/symmetric/risk_neutral/2p/2023-01-13 Fri 11:06:42",

    "/home/kohring/bnelearn/experiments/smooth-2023-uneven-batch-sizes/single_item/second_price/uniform/symmetric/risk_neutral/2p/2022-12-28 Wed 09:24:54",
    "/home/kohring/bnelearn/experiments/smooth-2023-uneven-batch-sizes/single_item/second_price/uniform/symmetric/risk_neutral/2p/2022-12-28 Wed 10:06:43",
    "/home/kohring/bnelearn/experiments/smooth-2023-uneven-batch-sizes/single_item/second_price/uniform/symmetric/risk_neutral/2p/2022-12-28 Wed 12:20:09",
    "/home/kohring/bnelearn/experiments/smooth-2023-uneven-batch-sizes/single_item/second_price/uniform/symmetric/risk_neutral/2p/2022-12-28 Wed 15:58:06",
    "/home/kohring/bnelearn/experiments/smooth-2023-uneven-batch-sizes/single_item/second_price/uniform/symmetric/risk_neutral/2p/2022-12-28 Wed 16:56:26",
    "/home/kohring/bnelearn/experiments/smooth-2023-uneven-batch-sizes/single_item/second_price/uniform/symmetric/risk_neutral/2p/2022-12-28 Wed 18:59:56",
    "/home/kohring/bnelearn/experiments/smooth-2023-uneven-batch-sizes/single_item/second_price/uniform/symmetric/risk_neutral/2p/2022-12-29 Thu 12:56:32",
    "/home/kohring/bnelearn/experiments/smooth-2023-uneven-batch-sizes/single_item/second_price/uniform/symmetric/risk_neutral/2p/2022-12-29 Thu 14:40:11",
    "/home/kohring/bnelearn/experiments/smooth-2023-uneven-batch-sizes/single_item/second_price/uniform/symmetric/risk_neutral/2p/2022-12-29 Thu 16:35:43",
    "/home/kohring/bnelearn/experiments/smooth-large-scale/main/single_item/second_price/uniform/symmetric/risk_neutral/2p/2023-01-14 Sat 19:16:07",
    "/home/kohring/bnelearn/experiments/smooth-large-scale/main-ESPGLearner/single_item/second_price/uniform/symmetric/risk_neutral/2p/2023-01-15 Sun 04:09:31",
    "/home/kohring/bnelearn/experiments/smooth-large-scale/main-PGLearner/single_item/second_price/uniform/symmetric/risk_neutral/2p/2023-01-15 Sun 03:49:05",
}

In [ ]:
aggregate_df = pd.DataFrame()
for i, path in enumerate(paths):
    df = multiple_exps_logs_to_df(
        path=path,
        metrics=metrics,
        precision=3,
        with_setting_parameters=True,
    )
    if df.Learner[0] == "ESPGLearner":
        learner = "NPGA"
    elif df.Learner[0] == "PGLearner":
        learner = "SM"
    elif df.Learner[0] == "ReinforceLearner":
        learner = "Reinforce"

    df = df[["Pricing", "Items", "Smoothing"] + [ALIASES_LATEX[m] for m in metrics]]
    df["Smoothing"] = df["Smoothing"].apply(lambda x: learner)
    df = df.rename(columns={"Smoothing": "Algo."})
    aggregate_df = pd.concat([aggregate_df, df])
aggregate_df.sort_values(["Pricing", "Items", "Algo."], inplace=True)
aggregate_df.set_index(["Pricing", "Item/home/kohring/bnelearn/experiments/smooth-large-scale/main-PGLearner/single_item/second_prices", "Algo."], inplace=True)
df_to_tex(aggregate_df, name='simultaneous_auction_table.tex',
          label='tab:simultaneous_auction', caption='Results of the simultaneous auction experiments.',
          index=True)
aggregate_df.head(30)

---
## 3. Temperature experiment

In [ ]:
configs = {
    "players":{
        "var_name": "n",
        "vars": [2, 3, 4],
    },
    "items":{
        "var_name": "m",
        "vars": [1, 2],
    },
    "batch_size":{
        "var_name": "h",
        "vars": [2**10, 2**14, 2**18, 2**22],
    },
}

In [ ]:
exp = "players"
# exp = "items"
# exp = "batch_size"
config = configs[exp]

In [ ]:
metrics = ['eval_vs_bne/L_2']
metric = "$L_2$"
path = f"/home/kohring/bnelearn/experiments/smooth-new/temperature-new-new/{exp}/single_item/first_price/uniform/symmetric/risk_neutral/"

In [ ]:
fig, ax = plt.subplots(figsize=(4.5, 4))

for i, var in enumerate(config['vars']):
    df = multiple_exps_logs_to_df(
        path=path + str(var) + "p" if exp == "players" else path + "2p",
        metrics=metrics,
        with_setting_parameters=True,
    )
    if len(df) > 0:
        if exp == "items":
            df = df[df["Units"] == var]
        elif exp == "batch_size":
            df = df[df["Batch"] == var]
        df.sort_values("Smoothing", inplace=True)
        df = df[df["Smoothing"] > 1e-05]

        smoothing = df.Smoothing.to_numpy()
        means, stds = split_mean_and_std_of_df_column(df, metric)


        smoothing_inter = np.arange(smoothing[0], smoothing[-1], 0.001)
        means_inter = interpolate.interp1d(smoothing, means, kind='quadratic')(smoothing_inter)
        stds_inter = interpolate.interp1d(smoothing, stds, kind='quadratic')(smoothing_inter)

        plt.plot(smoothing_inter, means_inter, "-", linewidth=1.7, color=COLORS[i])
        plt.fill_between(smoothing_inter, np.clip(means_inter-stds_inter, 0, 10), means_inter+stds_inter, color=COLORS[i], alpha=.1)
        plt.plot(smoothing, means, MARKERS[i], label=f"${config['var_name']} = {var}$", color=COLORS[i])
        # plt.fill_between(smoothing, np.clip(means-stds, 0, 10), means+stds, color=COLORS[i], alpha=.1)

        # plot optimum
        index_optimum = np.argmin(means_inter)
        # plt.plot(smoothing_inter[index_optimum], means_inter[index_optimum], "v", markersize=10, color=COLORS[i])

        if exp == "batch_size":
            print(f"Batch size = {var} (2**{round(np.log2(var))})\t|",
                f"optimum for lambda = {round(smoothing_inter[index_optimum], 4)}",
                f"with L_2 = {round(means_inter[index_optimum], 4)} "
                f"({round(stds_inter[index_optimum], 4)})")

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.grid(which='major', axis='y', alpha=.5)

plt.xlabel('temperature $\lambda$'); plt.ylabel(metric)
plt.xlim([-0.002, 0.0543]); plt.ylim([-0.003, 0.05])
plt.legend(loc='upper right', ncol=3, framealpha=1, edgecolor="white")
plt.tight_layout()
plt.savefig(path + f"temperature_{exp}_analysis.pdf")
plt.savefig(path + f"temperature_{exp}_analysis.png", dpi=300)
plt.show()

---
## 4. Ex post comparison

In [ ]:
valuation = 1
highest_opponent_bid = 0.5
t = 0.01

def utility(b):
    utility = np.zeros_like(b)
    utility[b > highest_opponent_bid] = valuation - b[b > highest_opponent_bid]
    return utility
    
def sm_utility(b):
    allocation = np.exp(b/t) / (np.exp(b/t) + np.exp(highest_opponent_bid/t))
    price = np.zeros_like(b)
    price[b > highest_opponent_bid] = b[b > highest_opponent_bid]
    price[b <= highest_opponent_bid] = highest_opponent_bid
    return (valuation - price)*allocation

In [ ]:
bids = np.linspace(0, 1, 200)

fig, ax = plt.subplots(figsize=(4.5, 3))
plt.plot(bids, utility(bids), label=f"$u_i({valuation}, b_i, 0.5)$", color=COLORS[0])
plt.plot(bids, sm_utility(bids), label="$u_i^{SM(" + str(t) + ")}" + f"({valuation}, b_i, 0.5)$", color=COLORS[1])
plt.xlabel("bid $b_i$"); plt.ylabel("ex post utility")

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.grid(which='major', axis='y', alpha=.5)

plt.xlim([0, 1]); plt.ylim([-0.05, .7])
plt.legend(loc='upper right', ncol=2, framealpha=1, edgecolor="white")
plt.tight_layout(); plt.savefig("ex-post-utility.pdf"); plt.savefig("ex-post-utility.png", dpi=300)
plt.show()